# 🧹 02. 정제(Cleansing) & 정규화(Normalization): 데이터 목욕시키기

## 1. 개요
토큰화로 문장을 잘라냈지만, 아직 **지저분한 것들**이 많이 묻어있습니다. 
분석에 방해되는 것들을 씻어내고(정제), 모양을 예쁘게 다듬는(정규화) 과정을 배워봅시다.

### 🔑 핵심 용어 사전
| 용어 | 뜻 | 비유 |
|:---:|:---:|:---:|
| **Noise (노이즈)** | 분석을 방해하는 방해꾼 데이터 | 흙 묻은 당근의 '흙' |
| **Cleansing (정제)** | 노이즈를 제거하는 작업 | 당근 씻기 |
| **Normalization (정규화)** | 표현 방법을 하나로 통일하는 작업 | 삐뚤빼뚤한 당근을 채썰기로 통일하기 |
| **Stopwords (불용어)** | 문법적으로 필요하지만 의미가 없는 단어 (은, 는, a, the) | 포장지 (내용물 아님) |

---

## 2. 정제와 정규화, 뭐가 다른가요?

| 구분 | 목표 | 예시 |
|:---:|:---:|:---|
| **정제 (Cleansing)** | **"더러운 것 버리기"** | 특수문자(`!@#`), 이모지(`😀`), 의미 없는 단어 삭제 |
| **정규화 (Normalization)** | **"같은 건 하나로 모으기"** | `US`와 `USA`는 같은 뜻이니 `USA`로 통일! `Goooooood` -> `Good` |

> **목표**: AI가 헷갈리지 않게 깔끔하고 통일된 데이터를 보여주는 것입니다.

## 3. 규칙 기반 정규화 (Rule-based Normalization)

"A가 나오면 B로 바꿔라!"라고 우리가 직접 규칙을 정해주는 방법입니다.
가장 단순하지만 효과가 확실합니다.

In [ ]:
# 예시 문장: 오타(Kimdom)와 약어(UK)가 섞여 있어서 지저분합니다.
text = 'The United Kimdom and UK have a long history together. Uh-oh! Something went wrong, uhoh'

# 1. replace 함수로 직접 바꿔주기
# 'United Kimdom' (오타) -> 'UK' (표준 단어)로 통합
# 'Uh-oh' -> 'uhoh' (표기 통일)
transformed_text = text.replace("United Kimdom", 'UK').replace('Uh-oh', 'uhoh')

print("원본:", text)
print("수정:", transformed_text)

### 대소문자 통합 (Case Folding)
영어에서 `Apple`과 `apple`은 사실 같은 사과입니다.
컴퓨터는 대문자와 소문자를 완전 다른 글자로 인식하기 때문에, 보통 **모두 소문자로** 바꿉니다.

In [ ]:
# .lower() 함수: 모든 영어를 소문자로 변신시킵니다.
lower_text = transformed_text.lower()
print(lower_text)

# 주의: 'US'(미국)를 'us'(우리)로 바꾸면 의미가 달라질 수 있으니 조심해야 합니다!

## 4. 정제 (Cleansing): 불필요한 단어 솎아내기

분석하는 데 도움이 안 되는 단어들을 과감하게 버립니다.

1. **등장 빈도가 너무 적은 단어**: 10만 개 글 중 딱 1번 나온 단어는 패턴을 찾을 수 없으니 버립니다.
2. **길이가 너무 짧은 단어**: 영어의 `a`, `I` 같은 건 큰 의미가 없을 때가 많습니다.
3. **불용어 (Stopword)**: `is`, `the`, `that` 처럼 문법적으론 중요하지만 뜻은 없는 단어를 버립니다.

### 4-1. 너무 드문 단어 지우기 (빈도수 기반)
`Counter` 도구를 쓰면 단어가 몇 번 나왔는지 세어줍니다.

In [ ]:
from nltk.tokenize import word_tokenize
from collections import Counter  # 개수 세주는 도구

# 예시 문장
text = "The quick brown fox jumps over the lazy dog. The fox is quick and agile"

# 1. 일단 단어로 쪼갭니다 (토큰화)
tokens = word_tokenize(text)

# 2. 개수를 셉니다
vocab = Counter(tokens)
print("단어별 개수:", vocab)

# 3. 2번 이상 나온 단어만 남겨봅시다.
filtered_tokens = []
for token in tokens:
    if vocab[token] >= 2:
        filtered_tokens.append(token)

print("------------------")
print("남은 단어들:", filtered_tokens)
# 'The', 'quick', 'fox' 처럼 2번 이상 나온 것만 살아남았습니다.

### 4-2. 너무 짧은 단어 지우기 (길이 기반)
영어에서는 길이가 1~2글자인 단어(a, at, in, on)들이 별로 중요하지 않을 때가 많습니다.

In [ ]:
# 단어 길이가 3글자 이상인 것만 남기기!
# 리스트 컴프리헨션(List Comprehension)이라는 파이썬 문법을 쓰면 한 줄로 가능합니다.

long_words = [word for word in tokens if len(word) >= 3]

print("3글자 이상인 단어들:", long_words)

### 4-3. 불용어(Stopwords) 제거하기
**"이 단어 리스트에 있는 건 다 지워!"** 라고 미리 정해둔 목록을 **불용어 사전**이라고 합니다.
NLTK에서 영어 불용어 사전을 제공해줍니다.

In [ ]:
import nltk
from nltk.corpus import stopwords

# 불용어 사전 다운로드
nltk.download('stopwords')

# 영어 불용어 목록 가져오기
stop_words_list = stopwords.words('english')

print(f"영어 불용어 개수: {len(stop_words_list)}개")
print("예시 10개:", stop_words_list[:10])
# i, me, my, myself... 같은 단어들이 들어있네요!

In [ ]:
# 문장에서 불용어 빼고 남기기
text = "The quick brown fox jumps over the lazy dog."
tokens = word_tokenize(text)

result = []
for word in tokens:
    # 소문자로 바꿔서 비교해야 정확합니다 (The != the)
    if word.lower() not in stop_words_list:
        result.append(word)

print("원본:", tokens)
print("불용어 제거 후:", result)
# 'The', 'over', 'the' 같은 단어들이 사라졌습니다!

## 5. 한국어 불용어 처리

한국어는 **조사(은/는/이/가/을/를)**나 **접속사(그리고/하지만)**, **의존명사(것/수/등)** 등을 주로 제거합니다.
한국어는 NLTK처럼 딱 정해진 불용어 사전이 없어서, 보통 개발자가 직접 리스트를 만들거나 인터넷에 공유된 리스트를 가져와서 씁니다.

In [ ]:
from konlpy.tag import Okt

okt = Okt()
korean_text = "이것은 예시 문장입니다. 그리고 저것은 테스트 문장입니다. 그저 예시일 뿐입니다."

# 1. 형태소 분석
morphs = okt.morphs(korean_text)
print("모든 형태소:", morphs)

# 2. 나만의 불용어 리스트 만들기
my_stopwords = ['은', '입니다', '.', '그리고', '저것', '그저', '일', '뿐']

# 3. 불용어 제거하기
clean_morphs = [word for word in morphs if word not in my_stopwords]

print("청소 끝난 형태소:", clean_morphs)

### 팁: 불용어 파일(.txt)로 관리하기

불용어 리스트가 수백 개가 넘어가면 코드에 다 적기 힘드니까,
`stopwords.txt` 같은파일에 저장해두고 불러와서 쓰는 게 좋습니다.

```python
# 파일 읽어서 리스트로 만드는 법 (참고용)
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()
```